In [2]:
import pickle
from pycocotools.coco import COCO
from collections import Counter
import os

cwd = os.getcwd()
print(cwd)

/home/shirota/Depth_image_captioning_pub/dataset


In [ ]:
# データの保存先
fp_train_caption = cwd+'/coco2014/captions_train2014.json'

fp_val_caption = cwd+'/coco2014/captions_val2014.json'

fp_word_to_id = cwd+'/coco2014/word_to_id.pkl'
fp_id_to_word = cwd+'/coco2014/id_to_word.pkl'

# キャプションを読み込み
train_coco = COCO(fp_train_caption)
train_anns_keys = train_coco.anns.keys()

val_coco = COCO(fp_val_caption)
val_anns_keys = val_coco.anns.keys()


# 単語ーID対応表の作成
train_coco_token = []
for key in train_anns_keys:
    caption = train_coco.anns[key]['caption']
    tokens = caption.lower().split()
    train_coco_token.extend(tokens)#キャプションの文字列が入ったリストを連結していく、[]+[]=[]

# validationセットから抽出
val_coco_token = []
for key in val_anns_keys:
    caption = val_coco.anns[key]['caption']
    tokens = caption.lower().split()
    val_coco_token.extend(tokens)

# ピリオド、カンマを削除
table = str.maketrans({'.': '',
                       ',': ''})
for k in range(len(train_coco_token)):
    train_coco_token[k] = train_coco_token[k].translate(table)

for k in range(len(val_coco_token)):
    val_coco_token[k] = val_coco_token[k].translate(table)

# 単語ヒストグラムを作成
train_coco_token.extend(val_coco_token)#trainとvalidationのトークンを連結
token = train_coco_token
freq = Counter(token)

# 3回以上出現する単語に限定して辞書を作成
vocab = [token for token, count in freq.items() if count >= 3]
sorted(vocab)

# 特殊トークンの追加
vocab.append('<start>') # 文章の始まりを表すトークンを追加
vocab.append('<end>')  # 文章の終わりを表すトークンを追加
vocab.append('<unk>')  # 辞書に無い単語を表すトークンを追加
vocab.append('<null>')  # 系列長を揃えるためのトークンを追加

# 単語ー単語ID対応表の作成
word_to_id = {token: i for i, token in enumerate(vocab)}
id_to_word = {i: token for i, token in enumerate(vocab)}

# ファイル出力
with open(fp_word_to_id, 'wb') as f:
    pickle.dump(word_to_id, f)
with open(fp_id_to_word, 'wb') as f:
    pickle.dump(id_to_word, f)

print(f'単語数: {str(len(word_to_id))}')